In [1]:
import pandas as pd
import requests
import json
import logging
from bs4 import BeautifulSoup
import time

In [2]:
# GET TOP TOURISM COUNTRIES
df_tourism = pd.read_csv('./data/most_traveled_countries.csv')
# SOURCE: http://worldpopulationreview.com/countries/most-visited-countries/
# df_tourism.head()

top20 = list(df_tourism.sort_values(by='touristArrivals', ascending=False)['name'][:20])
print(top20)

['France', 'United States', 'Spain', 'China', 'Italy', 'United Kingdom', 'Germany', 'Mexico', 'Thailand', 'Turkey', 'Austria', 'Malaysia', 'Hong Kong', 'Greece', 'Russia', 'Japan', 'Canada', 'Portugal', 'Saudi Arabia', 'Poland']


In [3]:
# GET COUNTRIES
country_names = [c_name for c_name in pd.read_csv('./data/country_names.csv')['name']]
# SOURCES: http://worldpopulationreview.com/countries/most-visited-countries/
    # & https://www.travel-advisory.info/api


In [4]:
len(country_names)

178

In [5]:
# GET ALL URLS FOR EACH COUNTRY
articles = []
for country in country_names:
#     print(f'Searching {country}...')
    for n in range(1, 4):
        try:
            url_base = f'https://amateurtraveler.com/tag/{country}+article/page/{n}/'
            res_ = requests.get(url_base)
            soup_ = BeautifulSoup(res_.content, 'lxml')
            time.sleep(1)

            for link in soup_.find_all(attrs={'class': 'category-post-desc'}):
                article = {}
                article['href'] = link.find('a').attrs['href']
                article['title'] = link.text.strip()
                article['country'] = country
                articles.append(article)
        except:
            print(f'Error with {country} at page {n}.')

df = pd.DataFrame(articles)
print('Shape of dataframe:', df.shape)
df.head()

Shape of dataframe: (316, 3)


,href,title,country
0,https://amateurtraveler.com/food-of-armenia/,Food of Armenia: The Tasty Side of Armenia,Armenia
1,https://amateurtraveler.com/visiting-history-f...,Visiting Historic Armenia,Armenia
2,https://amateurtraveler.com/graffiti-tour-of-b...,"Graffiti Tour of Buenos Aires, Argentina",Argentina
3,https://amateurtraveler.com/review-hotel-vitru...,"Review – Hotel Vitrum – Buenos Aires, Argentina",Argentina
4,https://amateurtraveler.com/iguazu-falls-argen...,"Iguazu Falls, Argentina – What You Need To Kno...",Argentina


In [7]:
# GET TEXT CONTENT FROM EACH POST
content_filed = []
for url in df['href']:
    try:
        res_post = requests.get(url)
        soup_post = BeautifulSoup(res_post.content, 'lxml')

        paragraphs = soup_post.find_all(attrs={'class': 'entry'})

        text = ' '.join([paragraph.text.replace('\n', '') for paragraph in paragraphs])
        content_filed.append(text)
    except:
        res_post = requests.get(f'https://amateurtraveler.com/{url}')
        soup_post = BeautifulSoup(res_post.content, 'lxml')

        paragraphs = soup_post.find_all(attrs={'class': 'entry'})

        text = ' '.join([paragraph.text.replace('\n', '') for paragraph in paragraphs])
        content_filed.append(text)
        
    time.sleep(1)
df['text'] = content_filed

In [8]:
df.tail()

,href,title,country,text
311,https://amateurtraveler.com/vietnam-visions/,Vietnam Visions,Vietnam,I’ve spent more time in Vietnam than any other...
312,https://amateurtraveler.com/africa-overland/,"Africa Overland – Tour from Johannesburg, Sout...",Zambia,Once a year I try and take a trip with a group...
313,https://amateurtraveler.com/zambia-road-trip/,Zambia Road Trip from Bottom to Top,Zambia,Self-driving through Zambia is an experience u...
314,https://amateurtraveler.com/africa-overland/,"Africa Overland – Tour from Johannesburg, Sout...",Zimbabwe,Once a year I try and take a trip with a group...
315,https://amateurtraveler.com/zimbabwe-safari-lo...,"Zimbabwe Safari Lodge – The Old Drift Lodge, L...",Zimbabwe,We sat in the lobby of the Old Drift Lodge whe...


In [9]:
df.to_csv('./data/articles_amateurtraveler.csv', index=False)